In [71]:
import pandas as pd
from pydantic import BaseModel, field_validator, ValidationError
import os




path = os.path.abspath('C:/Users/Balli/Pictures')

data_weblog = pd.read_csv(path +'/store_web_log.csv')

ip_info= pd.read_csv(path +'/ip_info.csv')

#perform a join querry of our two dataset that are related 
merged_data=pd.merge(data_weblog,ip_info, on='IP',how='left')

# converting to datetime to be able to get the date and hour of  daily transaction
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'],unit='s')

# filtering the date out of the datetime
merged_data['date'] = merged_data['timestamp'].dt.date

#filtering the hour of user session out of datetime
merged_data['hour'] = merged_data['timestamp'].dt.hour


grouped_Data=merged_data.groupby(['UserId','date','hour','Location','product_to_cart']).agg(
                                timespentdaily=('Duration','sum'),pages_visited=('Address','count')).reset_index()
print(grouped_Data)

FinalData=grouped_Data.drop_duplicates(subset=['UserId'])




# defining the customer web access information table

class customer_web_info_model(BaseModel):
    timespent_perday: int
    no_pages_visited: int
    location: str
    hour_session_starts: int
    was_there_purchase: int
    
    
    # creating validation for column number of pages visited
    @field_validator('no_pages_visited')
    def check_pages_visited(cls, value):
        if value <= 0:
            raise ValidationError('Pages visited field can not be less than 0')
        else:
            return value
        
   # creating validation for column was there a purchase to convert integer to a string answer of Yes or No 
    @field_validator('was_there_purchase')
    def purchase(cls, value):
        if value == 1:
            return 'Yes'
        else:
            return 'No'


# defining a variable list to store our data from the model customer_web_info       
data_list = []
# Iterating over the group_Data to load our model with data
for index, row in grouped_Data.iterrows():
    try:    
        dataTable1 = customer_web_info_model(timespent_perday=row['timespentdaily'] ,
                                     no_pages_visited=row['pages_visited'],
                                     location=row['Location'],hour_session_starts=row['hour'],
                                    was_there_purchase=row['product_to_cart']
                                     )
        
        data_list.append(dataTable1.model_dump())
    except Exception as error:
        print(f'error:{error}')

# converting data list to pandas dataframe
if data_list:
    df_data_list=pd.DataFrame(data_list)
    print(df_data_list)
    
    #saving our file/dataset as csv
    df_data_list.to_csv('Data Table 1.csv', index=False)
    print('Data Table saved successfully')















            UserId        date  hour                   Location  \
0   A1BG8QW55XHN6U  2020-07-28    19            Ontario, Canada   
1   A1BG8QW55XHN6U  2020-07-28    19            Ontario, Canada   
2   A1WMRR494NWEWV  2020-07-28    19       Texas, United States   
3   A1WMRR494NWEWV  2020-07-28    19       Texas, United States   
4   A22VW0P4VZHDE3  2020-07-28    19       Texas, United States   
5   A39HTATAQ9V7YF  2020-07-28    18  California, United States   
6   A39HTATAQ9V7YF  2020-07-28    18  California, United States   
7   A3JM6GV9MNOF9X  2020-07-28    18  California, United States   
8   A3JM6GV9MNOF9X  2020-07-28    18  California, United States   
9   A3V3RE4132GKRO  2020-07-28    19       Texas, United States   
10  A3V3RE4132GKRO  2020-07-28    19       Texas, United States   
11   AKJHHD5VEH7VG  2020-07-28    19        Ohio, United States   
12   AKJHHD5VEH7VG  2020-07-28    19        Ohio, United States   
13           guest  2020-07-28    18  California, United State

c:\Users\Balli\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [73]:
import pandas as pd
from pydantic import BaseModel, field_validator, ValidationError
from datetime import  datetime
import os

path = os.path.abspath('C:/Users/Balli/Pictures')

productData= pd.read_csv(path +'/store_product_table.csv')

transactionData= pd.read_csv(path +'/store_transaction_records.csv')

#performing a join querry of thw two dataset related by transaction
merged_data1 = pd.merge(transactionData,productData, on='ProductID',how='left')

# converting to datetime to be able to get the date and hour of  daily transaction
merged_data1['transactn_time'] = pd.to_datetime(merged_data1['time_transactn'],unit='s')

# filtering the date out of the datetime
merged_data1['date'] = merged_data1['transactn_time'].dt.date

#filtering the hour of transaction out of datetime
merged_data1['hour'] = merged_data1['transactn_time'].dt.hour

#creating a dictionary of our new formated dataset
DictData={'date':merged_data1['date'],'houroftransactn':merged_data1['hour'],
            'productname':merged_data1['Productdescriptn'],'price':merged_data1['price']}

#converting to pandas dataframe
finaldata1=pd.DataFrame(DictData)
print(finaldata1.head(10))




#Defining the transacion table or model

class transaction_model(BaseModel):
    productname:str
    price: float
    date: datetime
    hour_of_transaction: int

    @field_validator('price')
    def check_price(cls, value):
        if value:
            return f'${value}'
        elif value <= 0:
            raise ValidationError
        else:
            return None

    
datalist_2 = []
# iterating over finaldata1 to load our model
for index, row in finaldata1.iterrows():
    dataTable2 = transaction_model(productname=row['productname'],date= row['date'], hour_of_transaction=row['houroftransactn'], 
                                   price=row['price'])
    
    datalist_2.append(dataTable2.model_dump())
# converting datalist_2 to pandas dataframe
if datalist_2:
        datalist_2display=pd.DataFrame(datalist_2)
        print(datalist_2display)
        # saving our data table as csv
        datalist_2display.to_csv('Data Table 2.csv', index=False)
        print('Data Table saved successfully')




         date  houroftransactn   productname  price
0  2020-07-28               18     Red Pants  10.99
1  2020-07-28               19     Red Pants  10.99
2  2020-07-28               20  Flower Dress  25.99
3  2020-07-28               20   Orange Belt  13.99
4  2020-07-28               20   Pink Shorts   9.99
5  2020-07-28               21     Red Pants  10.99
6  2020-07-28               21  Flower Dress  25.99
7  2020-07-28               21   Orange Belt  13.99
8  2020-07-28               21  Yellow Skirt  17.99
9  2020-08-02                8     Red Pants  10.99
     productname   price       date  hour_of_transaction
0      Red Pants  $10.99 2020-07-28                   18
1      Red Pants  $10.99 2020-07-28                   19
2   Flower Dress  $25.99 2020-07-28                   20
3    Orange Belt  $13.99 2020-07-28                   20
4    Pink Shorts   $9.99 2020-07-28                   20
5      Red Pants  $10.99 2020-07-28                   21
6   Flower Dress  $25.99 2020

c:\Users\Balli\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [74]:
import pandas as pd
from pydantic import BaseModel, field_validator, ValidationError
from datetime import  datetime
import os
#defining the path to our save dataset file
path = os.path.abspath('C:/Users/Balli/Pictures')

data_weblog = pd.read_csv(path +'/store_web_log.csv')


# Filter rows where Link is null
last_page_visits = data_weblog[data_weblog['Link'].isnull()]
#print(last_page_visits)

# Group by the Address column and count occurrences
last_page_counts = last_page_visits.groupby('Address').size().reset_index(name='count')




# defining last  web page visited  count model


class last_page_visited(BaseModel):
    last_web_pagevisited: str
    no_of_time_pagevisited: int

#set a list variable to store our table data
datatable3 = []

#iterate over the last_page_count table and load into our model
for index, row in last_page_counts.iterrows():
    Data3=last_page_visited(last_web_pagevisited=row['Address'],no_of_time_pagevisited=row['count'])
    datatable3.append(Data3.model_dump())
#convert our data table 3 into pandas dataframe
if datatable3:
    df_datatable3=pd.DataFrame(datatable3)
    print(df_datatable3)
    # saving our file/data  as csv
    df_datatable3.to_csv('Data Table 3.csv', index=False)
    print('Data Table saved successfully')


                last_web_pagevisited  no_of_time_pagevisited
0  www.store.com/checkout_trans#1543                       1
1  www.store.com/checkout_trans#1547                       1
2  www.store.com/checkout_trans#1548                       1
3       www.store.com/item#558925278                       1
4                www.store.com/main#                       1
5    www.store.com/ty#A39HTATAQ9V7YF                       1
6     www.store.com/ty#AKJHHD5VEH7VG                       1
Data Table saved successfully


In [10]:
import pandas as pd
from pydantic import BaseModel, field_validator, ValidationError
import phonenumbers
import os
import numpy as np
import random


path = os.path.abspath('C:/Users/Balli/Pictures')

customerdataset= pd.read_csv(path +'/customerdata.csv')
#print(customerdataset)

# The Gender column is empty, and since there is no module in python currently to decipher
# male or female from the names. it became sacrosanct to use Excel and manually fill this column since
# we can decipher male names from female names by bare looking at it.


#customerdataset.info()
#customerdataset.duplicated()
#customerdataset.loc[customerdataset.duplicated(),:]


# now let us deal wit outliers
# from the dataset it can be noticed that the age column have an outlier which is 500, because no human in the generation
# can live to 500 years

# because this dataset is no many and we can easily spot outliers so we may not need to use IQR method but directly
# replace the outlier with the mean or median or mode as the case may be, but for this dataset, i may have to use mean
mean_age=customerdataset['age'].mean()
mode_age=customerdataset['age'].mode()[0]
median_age=customerdataset['age'].median()

# Now i am going t replace the outlier with the mean_age

customerdataset.loc[(customerdataset['age'] >= 500),'age']=mean_age

# replacing the null value in the age column with the median age
customerdataset['age'].fillna(median_age,inplace=True)

# Age does not have decimal that is there is no half age. so we need to convert the age dtype to proper int
customerdataset['age'] = customerdataset['age'].astype(int)

#  Now cleaning the Education column and replacing the null with the most frequent education category

customerdataset['education'].value_counts(dropna=False)
customerdataset['education'].fillna(value='College',inplace=True)

# stripping off excess whitespaces in the Education column
customerdataset['education']=customerdataset['education'].str.strip()

# Now cleaning the income column to replace the null with the mean income and adding the $ sign

mean_income=round(customerdataset['income'].mean(),2)
customerdataset['income'].fillna(mean_income,inplace=True)


def add_currencySign(income):
    if income:
        return '${:,}'.format(income)
    else:
        return None
    
customerdataset['income']=customerdataset['income'].apply(add_currencySign)


# Now cleaning the Consent column to replace any string to integer and convert datatype to int
customerdataset.loc[(customerdataset['consent'] == 'i'),'consent'] = 1
customerdataset['consent']=customerdataset['consent'].astype(int)

# Cleaning the pone number column and also creating a link for one-time click dial

# Defining a function to clean and replace null phone column with a random generated number as phone

def formated_phone(phone):
        if pd.isnull(phone):
          return  int(random.randint(10**9,10**10))
        
        else:
            return int(phone)
 # Appling the function           
customerdataset['phone']=customerdataset['phone'].apply(formated_phone)

# Filling any phone number that is less 10 digit
customerdataset['phone']=customerdataset['phone'].astype(str).str.zfill(10)

# Trimming any phone number that is greater than 10 digits
customerdataset['phone'] = customerdataset['phone'].astype(str).str.slice(0,9)

# Now converting the phone number column to proper phone number standard using the python phonenumber module

def format_phone_number(phone):
    try:
        # Parse the phone number
        
        parsed_number = phonenumbers.parse(phone, "CA")
        
        # Format the phone number in E.164 format
        formatted_number = phonenumbers.format_number(parsed_number, phonenumbers.PhoneNumberFormat.E164)
        
        return formatted_number
    
    except phonenumbers.NumberParseException:
        return phone  # Return the original phone number if parsing fails
    
# Now applying the function format_phone_number to the phone column of our dataset
customerdataset['phone']=customerdataset['phone'].apply(format_phone_number)

# Final step is to create a link on the phone column so that this can aid auto-dail by creating a function

def clickable_phone(phone):
    if phone:
        return f'<a href="Tel:{phone}">{phone}</a>'
    else:
        return None

customerdataset['phone']=customerdataset['phone'].apply(clickable_phone)

print(customerdataset)

# To enable the customer rep to dial customer phone numbers automatically, we need to save te dataset in an html format
# this will enable the dataset to e opened through a browser and then giving the link to auto dial a number

customerdataset.to_html('customerdataset.html',escape=False,index=False)
print('dataset saved in html file successfully')










        name                                      phone  age       income  \
0       Thom  <a href="Tel:+1604907178">+1604907178</a>   67   $754,298.0   
1       Thom  <a href="Tel:+1908876453">+1908876453</a>   23     $9,000.0   
2      Colin  <a href="Tel:+1604675894">+1604675894</a>   54    $62,840.0   
3      Colin  <a href="Tel:+1604675894">+1604675894</a>   54    $62,840.0   
4       Phil  <a href="Tel:+1509524718">+1509524718</a>   12    $78,000.0   
5         Ed  <a href="Tel:+1509826727">+1509826727</a>   35   $445,566.0   
6     Johnny  <a href="Tel:+1604918675">+1604918675</a>   87   $212,121.0   
7      Sarah  <a href="Tel:+1402289101">+1402289101</a>   65    $36,000.0   
8       Yoda  <a href="Tel:+1642098256">+1642098256</a>   62    $49,382.0   
9     Alexis  <a href="Tel:+1604897178">+1604897178</a>   44    $12,344.0   
10     Hanna  <a href="Tel:+1509786098">+1509786098</a>   32     $1,010.0   
11    Rachel  <a href="Tel:+1709878923">+1709878923</a>   28   $100,000.0   

C:\Users\Balli\AppData\Local\Temp\ipykernel_6276\3237296706.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customerdataset['age'].fillna(median_age,inplace=True)
C:\Users\Balli\AppData\Local\Temp\ipykernel_6276\3237296706.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F